### Code Cell 1 (5%) - Import Required Libraries and Load Data

In [10]:
import numpy as numpy
import pandas as pd

In [11]:
df = pd.read_csv("/Users/rhy/Library/CloudStorage/OneDrive-ArizonaStateUniversity/ASU/SEM 2/CIS 509 Analytics Unstructured Data/Lab Assignment 4/restaurant_reviews_az.csv")[:2000]
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,IVS7do_HBzroiCiymNdxDg,fdFgZQQYQJeEAshH4lxSfQ,sGy67CpJctjeCWClWqonjA,3,1,1,0,"OK, the hype about having Hatch chili in your ...",2020-01-27 22:59:06
1,QP2pSzSqpJTMWOCuUuyXkQ,JBLWSXBTKFvJYYiM-FnCOQ,3w7NRntdQ9h0KwDsksIt5Q,5,1,1,1,Pandemic pit stop to have an ice cream.... onl...,2020-04-19 05:33:16
2,oK0cGYStgDOusZKz9B1qug,2_9fKnXChUjC5xArfF8BLg,OMnPtRGmbY8qH_wIILfYKA,5,1,0,0,I was lucky enough to go to the soft opening a...,2020-02-29 19:43:44
3,E_ABvFCNVLbfOgRg3Pv1KQ,9MExTQ76GSKhxSWnTS901g,V9XlikTxq0My4gE8LULsjw,5,0,0,0,I've gone to claim Jumpers all over the US and...,2020-03-14 21:47:07
4,Rd222CrrnXkXukR2iWj69g,LPxuausjvDN88uPr-Q4cQA,CA5BOxKRDPGJgdUQ8OUOpw,4,1,0,0,"If you haven't been to Maynard's kitchen, it'...",2020-01-17 20:32:57


In [12]:
df.shape

(2000, 9)

### Code Cell 2 (5%) - Data Preprocessing

In [13]:
df = df[df['stars'] != 3]

df['Sentiment'] = df['stars'].apply(lambda x: 1 if x >= 4 else 0)

positive_reviews = df[df['Sentiment'] == 1].sample(n=50, random_state=1)
negative_reviews = df[df['Sentiment'] == 0].sample(n=50, random_state=1)

df1 = pd.concat([positive_reviews, negative_reviews]).reset_index(drop=True)
df1.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Sentiment
0,W2AtefJfe983yRvXfHxGhg,meQCVrrJxwdjS_1BFooTUg,gMpYdAe1lZWDuBRw2bPEhg,5,0,0,0,My first time trying their food. Absolutely de...,2021-04-20 01:53:18,1
1,9IN97k-8LG5wrbGbp4GG5A,BxFQzm3dB8SVVpITk7Z7jQ,wJmyu7W1K9A_gE8Ed4Bc9w,5,1,0,1,Such a wonderful dinning experience!! Went int...,2020-03-03 19:06:59,1
2,Z9cSDkJ0J00o25bZxRCbSg,yy81U2zRyL5QEdj8faja1Q,V9XlikTxq0My4gE8LULsjw,4,0,0,0,These are really tough times (June 2020) for r...,2020-06-21 17:28:55,1
3,jxQ_Z5NDguWsvGO8_o0wKw,LGb9UQFdW9PkDHEUyg30lw,UCMSWPqzXjd7QHq7v8PJjQ,5,1,0,0,"Extensive, varied, and delicious menu with ple...",2021-01-02 16:48:42,1
4,fHQUBXyZ5qKIK_3EsUo3sw,eHFpMUNb1_FlcK-Dl_9cMw,3StNEgKAwpCFR1q0urmJrw,5,0,0,0,"Best ramen place in Tucson, from what I've had...",2020-03-08 20:21:29,1


In [14]:
df1.shape

(100, 10)

### Code Cell 3 (20%) - Perform Sentiment Analysis Using Zero-Shot Learning

In [ ]:
from transformers import pipeline

# Load the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define the candidate labels
candidate_labels = ["positive", "negative"]

# Perform zero-shot classification on the reviews
df1['ZeroShotSentiment'] = df1['text'].apply(lambda x: classifier(x, candidate_labels)['labels'][0])

df1.head()

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use mps:0


,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Sentiment,ZeroShotSentiment
0,W2AtefJfe983yRvXfHxGhg,meQCVrrJxwdjS_1BFooTUg,gMpYdAe1lZWDuBRw2bPEhg,5,0,0,0,My first time trying their food. Absolutely de...,2021-04-20 01:53:18,1,positive
1,9IN97k-8LG5wrbGbp4GG5A,BxFQzm3dB8SVVpITk7Z7jQ,wJmyu7W1K9A_gE8Ed4Bc9w,5,1,0,1,Such a wonderful dinning experience!! Went int...,2020-03-03 19:06:59,1,positive
2,Z9cSDkJ0J00o25bZxRCbSg,yy81U2zRyL5QEdj8faja1Q,V9XlikTxq0My4gE8LULsjw,4,0,0,0,These are really tough times (June 2020) for r...,2020-06-21 17:28:55,1,negative
3,jxQ_Z5NDguWsvGO8_o0wKw,LGb9UQFdW9PkDHEUyg30lw,UCMSWPqzXjd7QHq7v8PJjQ,5,1,0,0,"Extensive, varied, and delicious menu with ple...",2021-01-02 16:48:42,1,positive
4,fHQUBXyZ5qKIK_3EsUo3sw,eHFpMUNb1_FlcK-Dl_9cMw,3StNEgKAwpCFR1q0urmJrw,5,0,0,0,"Best ramen place in Tucson, from what I've had...",2020-03-08 20:21:29,1,positive
